In [47]:
import pandas as pd

df = pd.read_csv('train.csv')
df.drop(columns=['ID'], inplace=True)

print(df.head())

    timestamp item corporation location  supply(kg)  price(원/kg)
0  2019-01-01   TG           A        J         0.0          0.0
1  2019-01-02   TG           A        J         0.0          0.0
2  2019-01-03   TG           A        J     60601.0       1728.0
3  2019-01-04   TG           A        J     25000.0       1408.0
4  2019-01-05   TG           A        J     32352.0       1250.0


In [48]:
df['year'] = pd.to_datetime(df['timestamp']).dt.year
df['month'] = pd.to_datetime(df['timestamp']).dt.month
df['day'] = pd.to_datetime(df['timestamp']).dt.day
df_encoded = pd.get_dummies(df, columns=['item', 'corporation', 'location'])
df_encoded.drop(columns=['timestamp'], inplace=True)
df= df_encoded
print(df.columns)

Index(['supply(kg)', 'price(원/kg)', 'year', 'month', 'day', 'item_BC',
       'item_CB', 'item_CR', 'item_RD', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'corporation_F', 'location_J', 'location_S'],
      dtype='object')


In [49]:
col_list = df.columns
col_list = [col for col in col_list if col != 'price(원/kg)' and col != 'supply(kg)']

In [50]:
print(df)

       supply(kg)  price(원/kg)  year  month  day  item_BC  item_CB  item_CR  \
0             0.0          0.0  2019      1    1    False    False    False   
1             0.0          0.0  2019      1    2    False    False    False   
2         60601.0       1728.0  2019      1    3    False    False    False   
3         25000.0       1408.0  2019      1    4    False    False    False   
4         32352.0       1250.0  2019      1    5    False    False    False   
...           ...          ...   ...    ...  ...      ...      ...      ...   
59392    452440.0        468.0  2023      2   27    False    False    False   
59393    421980.0        531.0  2023      2   28    False    False    False   
59394    382980.0        574.0  2023      3    1    False    False    False   
59395    477220.0        523.0  2023      3    2    False    False    False   
59396    427520.0        529.0  2023      3    3    False    False    False   

       item_RD  item_TG  corporation_A  corporation

In [51]:
X = df[col_list]
Y = df['price(원/kg)']

split_index = int(len(X) * 0.9)
X_train, X_test = X[:split_index], X[split_index:]
Y_train, Y_test = Y[:split_index], Y[split_index:]

In [52]:
import xgboost as xgb
model1_xg = xgb.XGBRegressor()
model1_xg.fit(X_train, Y_train)

from sklearn.metrics import mean_squared_error, mean_absolute_error

predictions = model1_xg.predict(X_test)
mse = mean_squared_error(Y_test, predictions)

print("Mean Squared Error: ", mse)

Mean Squared Error:  1040065.2895684019


In [53]:
df2 = pd.read_csv('test.csv')
df2.drop(columns=['ID'], inplace=True)
df2['year'] = pd.to_datetime(df2['timestamp']).dt.year
df2['month'] = pd.to_datetime(df2['timestamp']).dt.month
df2['day'] = pd.to_datetime(df2['timestamp']).dt.day
df2_encoded = pd.get_dummies(df2, columns=['item', 'corporation', 'location'])
df2_encoded.drop(columns=['timestamp'], inplace=True)
df2= df2_encoded


In [54]:
X2 = df2[col_list]

model2_xg = xgb.XGBRegressor()
model2_xg.fit(X, Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [55]:
len(X2)

1092

In [56]:
predictions = model1_xg.predict(X2)
len(predictions)

1092

In [59]:
print(predictions)

[3468.6318  2166.1438  3007.6904  ...  346.06494 1047.3846  1095.4005 ]


In [60]:
submission = pd.read_csv('sample_submission.csv')

# 'answer' 열에 예측값 리스트 할당
submission['answer'] = predictions

# 수정된 데이터프레임을 'sample_submission.csv' 파일로 저장
submission.to_csv('sample_submission.csv', index=False)